In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import *
from keras import regularizers
from keras import backend as K
import matplotlib.pyplot as plt

num_classes = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

In [ ]:
#base model + weight decay + drop out
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_train.shape[1:], kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(1e-3),
              metrics=['accuracy'])

In [3]:
nfold = 10
batch_size = 128
epochs = 100

pred = np.zeros((len(x_test), 10))  #(10000,10)

history = []
    
kf = KFold(n_splits=nfold, shuffle=True, random_state=520).split(x_train, y_train)
for i, (train_fold, valid_fold) in enumerate(kf):
    print('##### ' + str(i+1) + 'fold #####')
    
    X_train, X_valid = x_train[train_fold, :], x_train[valid_fold, :] #(45000, 32, 32, 3) (5000, 32, 32, 3)
    Y_train, Y_valid = y_train[train_fold, :], y_train[valid_fold, :] #(45000, 10)        (5000, 10)
    
    datagen = ImageDataGenerator(horizontal_flip=True)
    datagen.fit(X_train)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='auto')
    plateau = ReduceLROnPlateau(monitor="val_loss", verbose=1, mode='auto', factor=0.5, patience=2)
    checkpoint = ModelCheckpoint('./model/' + str(i) + '.hdf5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto',save_weights_only=True)
    
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_train.shape[1:], kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(1e-3),
                  metrics=['accuracy'])

    train_history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                                        epochs=epochs,
                                        validation_data=(X_valid, Y_valid),
                                        callbacks=[early_stopping, plateau, checkpoint]
                                       )
    history.append(train_history.history)
    model.load_weights('./model/' + str(i) + '.hdf5')
    
    pred += model.predict(x_test)
    K.clear_session()

##### 1fold #####

Epoch 1/100
352/352 [==============================] - 11s 30ms/step - loss: 1.7617 - accuracy: 0.3922 - val_loss: 1.4303 - val_accuracy: 0.5106

Epoch 00001: val_loss improved from inf to 1.43032, saving model to ./model/0.hdf5
Epoch 2/100
352/352 [==============================] - 9s 25ms/step - loss: 1.3182 - accuracy: 0.5591 - val_loss: 1.1584 - val_accuracy: 0.6228

Epoch 00002: val_loss improved from 1.43032 to 1.15837, saving model to ./model/0.hdf5
Epoch 3/100
352/352 [==============================] - 9s 25ms/step - loss: 1.1512 - accuracy: 0.6313 - val_loss: 1.0121 - val_accuracy: 0.6822

Epoch 00003: val_loss improved from 1.15837 to 1.01215, saving model to ./model/0.hdf5
Epoch 4/100
352/352 [==============================] - 9s 25ms/step - loss: 1.0711 - accuracy: 0.6669 - val_loss: 0.9506 - val_accuracy: 0.7084

Epoch 00004: val_loss improved from 1.01215 to 0.95061, saving model to ./model/0.hdf5
Epoch 5/100
352/352 [==============================] - 9

In [4]:
print(accuracy_score(np.argmax(pred, 1), np.argmax(y_test, 1)))

0.8609
